# MODUL PINA (Python 3.12)

# Vorhersage des Ödometerversuches implementiert
Ziel war die Implementierung eines neuronalen Netzwerks zur Modellierung des Ödometerversuchs. Dabei wurden gegebene Input-Parameter verarbeitet, um Output-Parameter vorherzusagen. Die physikalischen Rahmenbedingungen wurden zunächst auf Null gesetzt, sodass das Modell ausschließlich auf der KI-basierten Struktur arbeitet, ohne physikalische Optimierungen durch Physical Informed Neural Networks (PINNs).
<br>
Diese grundlegende Umsetzung bildet die Basis für weiterführende Optimierungen, wie die Integration physikalischer Gesetzmäßigkeiten, die jedoch nicht Teil des initialen Arbeitsauftrags waren.

#### Das Problem ist wie folgt definiert:
$$
\begin{array}{rcl}
    \sigma_{t+1} & = & \sigma_{t}+\Delta\sigma \\ \\
    \Delta\sigma & = & E_s\cdot \Delta\epsilon \\ 
    E_s & = & \frac{1+e_0}{C_c} \cdot \sigma_t
\end{array}
\hspace{2cm}
\begin{array}{l}
    \textbf{Annahmen:} \\ \\
    \text{Startwert d. Iteration: } \sigma_t = 1,00 \\ 
    e_0 = 1,00 \\ 
    C_c = 0,005 \\
    \Delta\epsilon = 0,0005
\end{array}
$$

<br> 

Um das PINA-Model zu testen werden wir folgende vorberechnete Werte verwenden: `Input` { $\sigma_t$ }, `Output` { $E_s$ }.
<br>
### Variablendeklaration
- $\sigma_t$ = `sigma_t`
- $\Delta\epsilon$ = `delta_epsilon`
- $\sigma_{t+1}$ = `delta_sigma
- $E_s$ = `e_s`

# Generating random trainings data

In [1]:
from random import randint

# Define input and output parameters
input_str = "sigma_t"
output_str = "e_s"

# 0 : simple, 1 : improved
oedo_model = 0

# Defining problem parameters
delta_epsilon = 0.0005
C_c = 0.005
e_0 = 1.0
amount_trainings_data = 100

# Data preparation for
oedo_para = {
    "max_n": 1,
    "e_0": e_0,
    "C_c": C_c,
    "delta_epsilon": delta_epsilon,
}

# Load problem and generate  data from 00_problem_settings_functions.ipynb

Available classes: `Oedometer` <br>
Returns `list_input` and `list_output` as type `list` <br>
Returns `tensor_input` and `tensor_output` as type `tensor`

In [2]:
from handler.handleData import generate_data
from random import sample

if oedo_model == 0:
    from classes.classOedometerSimple import Oedometer
else:
    from classes.classOedometerSimple import Oedometer

sigma_t_train = [1] + sample(range(0, amount_trainings_data * 2), amount_trainings_data)

list_input, list_output, tensor_input, tensor_output = generate_data(oedo=oedo_para, oedo_class=Oedometer, sigma_t=sigma_t_train, amount_trainings_data=amount_trainings_data)

# Show trainingsdata (List) as DataFrame
Type `list`: `list_input` and `list_output`

In [3]:
import pandas as pd
from pandas import DataFrame

pd.DataFrame([[input_str] + list_input, [output_str] + list_output])

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,sigma_t,1.0,139.0,81.0,15.0,112.0,64.0,193.0,57.0,45.0,...,136.0,138.0,40.0,33.0,186.0,78.0,48.0,2.0,85.0,59.0
1,e_s,400.0,55600.0,32400.0,6000.0,44800.0,25600.0,77200.0,22800.0,18000.0,...,54400.0,55200.0,16000.0,13200.0,74400.0,31200.0,19200.0,800.0,34000.0,23600.0


# Show trainingsdata (Tensor) as DataFrame
Type `tensor`: `tensor_input` and `tensor_output`

In [4]:
tensor_data_df = pd.DataFrame(torch.cat((tensor_input, tensor_output), dim=1), columns = [input_str, output_str])
tensor_data_df

NameError: name 'torch' is not defined

## Tensor to LabelTensor for PINA

In [5]:
from pina.utils import LabelTensor

label_tensor_input = LabelTensor(tensor_input,[input_str])
label_tensor_output = LabelTensor(tensor_output, [output_str])

# Show trainingsdata (LabelTensor) as DataFrame
Type `LabelTensor`: `label_tensor_input` and `label_tensor_output`

In [6]:
tensor_input_df = pd.DataFrame(torch.cat((label_tensor_input, label_tensor_output), dim=1), columns = [input_str, output_str])

print('Input Size: ', label_tensor_input.size())
print('Output Size: ', label_tensor_output.size(), '\n')
tensor_input_df

Input Size:  torch.Size([100, 1])
Output Size:  torch.Size([100, 1]) 



,sigma_t,e_s
0,30.0,12000.0
1,43.0,17200.0
2,22.0,8800.0
3,48.0,19200.0
4,14.0,5600.0
...,...,...
95,25.0,10000.0
96,43.0,17200.0
97,27.0,10800.0
98,24.0,9600.0


### **Definition eines PINN-Problems in PINA**  

In [7]:
from pina.problem import AbstractProblem
from pina.domain import CartesianDomain
from pina import Condition

input_conditions = {'data': Condition(input=label_tensor_input, target=label_tensor_output),}

class SimpleODE(AbstractProblem):

    # Definition der Eingabe- und Ausgabevariablen basierend auf LabelTensor
    input_variables = label_tensor_input.labels
    output_variables = label_tensor_output.labels

    # Wertebereich
    domain = CartesianDomain({label_tensor_input: [0, 1]})#, 'delta_epsilon': [0, 1]})  # Wertebereich immer definieren!

    # Definition der Randbedingungen und (hier: nur) vorberechnetet Punkte
    conditions = input_conditions

    label_tensor_output=label_tensor_output

    # Methode zur Definition der "wahren Lösung" des Problems
    def truth_solution(self, pts):
        return torch.exp(pts.extract(label_tensor_input))

# Problem-Instanz erzeugen
problem = SimpleODE()

print('Input: ', problem.input_variables)
print('Output: ', problem.output_variables)

Input:  ['sigma_t']
Output:  ['e_s']


# Training eines Physics-Informed Neural Networks (PINN) mit PINA

In [12]:
from pina import Trainer
from pina.solver import PINN
from pina.model import FeedForward
from pina.callback import MetricTracker
import torch.nn as nn
# Model erstellen
model = FeedForward(
    layers=[1,2,1],
    func=nn.ReLU,
    output_dimensions=len(problem.output_variables),
    input_dimensions=len(problem.input_variables)
)

# PINN-Objekt erstellen
pinn = PINN(problem, model)

# Trainer erstellen mit TensorBoard-Logger
trainer = Trainer(
    solver=pinn,
    max_epochs=1000,
    callbacks=[MetricTracker()],
    batch_size=16,
    accelerator='cpu',
    enable_model_summary=False,
)


# Training starten
trainer.train()

print('\nFinale Loss Werte')
# Inspect final loss
trainer.logged_metrics

TypeError: empty() received an invalid combination of arguments - got (tuple, dtype=NoneType, device=NoneType), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (tuple of ints size, *, torch.memory_format memory_format = None, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)


In [ ]:
import matplotlib.pyplot as plt

data_loss = trainer.callbacks[0].metrics["train_loss_epoch"].tolist()

plt.plot(data_loss, label="Loss")
plt.xlabel('Epochs')
plt.ylabel('Train Loss')
plt.show()

# Plot of stress–strain curve

In [ ]:
from handler.handleVisuals import plot_result_graph

oedo_para = {
        "max_n": 50,
        "e_0": e_0,
        "C_c": C_c,
        "delta_epsilon": delta_epsilon,
        "sigma_t": 1,
    }

oedo = Oedometer(**oedo_para)
e_s_list, delta_sigma_list = plot_result_graph(model=model, oedo=oedo,iterations=oedo_para["max_n"], start_sigma=1,delta_epsilon=0.0005)

In [ ]:
from handler.handleVisuals import plot_result_dataframe
from IPython.display import Markdown, display

plot_result_dataframe(pd, e_s_list, delta_sigma_list)